# Data Export for (https://github.com/albblgb/Deep-Steganalysis.git)

In [9]:
import os
import sys

import shutil

import numpy as np
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import retreive_datasets
from model_xray.config_classes import *
from PIL import Image

from model_xray.zenml.pipelines.data_creation.dataset_compilation import compile_and_save_preprocessed_images_dataset_pipeline
from model_xray.options import *
from model_xray.config_classes import *

curr_mc = 'famous_le_10m'

train_mzs, test_mzs = dataset_split[curr_mc]

imsize = 256
imtype = ImageType.GRAYSCALE_THREEPART_WEIGHTED_AVG

image_rep_config = ImageRepConfig.ret_image_rep_config(imtype)

image_preprocess_config=ImagePreprocessConfig(
    image_height=imsize,
    image_width=imsize,
)

from model_xray.utils.dataset_utils import *

pretrained_model_train_configs = get_pretrained_model_configs(train_mzs)
pretrained_model_test_configs = get_pretrained_model_configs(test_mzs)

x_train = 15

test_benigns_dataset_name = get_dataset_name(
    mc=curr_mc,
    xs=[None,],
    imsize=imsize,
    imtype=imtype,
    ds_type='test'
)

test_attacked_dataset_name = get_dataset_name(
    mc=curr_mc,
    xs=[x_train,],
    imsize=imsize,
    imtype=imtype,
    ds_type='test'
)

trainset_name = get_dataset_name(
    mc=curr_mc,
    xs=[None,x_train],
    imsize=imsize,
    imtype=imtype,
    ds_type='train'
)



trainset_names = [trainset_name,]

testset_names = [
    test_benigns_dataset_name, test_attacked_dataset_name
]

requested_datasets = trainset_names + testset_names

datasets = retreive_datasets(requested_datasets)

def output_imgs(img_arrs, path, output_as_rgb=False):
    os.makedirs(path, exist_ok=True)
    for i, img_arr in enumerate(img_arrs):
        img = Image.fromarray(img_arr)
        if output_as_rgb:
            img = img.convert('RGB')
        img.save(f'{path}/{i+1}.png')

def output_dataset(X,y, benign_path, mal_path, reset_dirs=False):
    if reset_dirs:
        if os.path.exists(benign_path):
            shutil.rmtree(benign_path)
        if os.path.exists(mal_path):
            shutil.rmtree(mal_path)

    os.makedirs(benign_path, exist_ok=True)
    os.makedirs(mal_path, exist_ok=True)

    X_benign = X[y == PreprocessedImageDatasetLabel.BENIGN]
    output_imgs(X_benign, benign_path)
    X_mal = X[y == PreprocessedImageDatasetLabel.ATTACKED]
    output_imgs(X_mal, mal_path)

def merge_datasets(datasets):
    Xs = []
    ys = []
    for X,y in datasets:
        Xs.append(X)
        ys.append(y)
    return np.vstack(Xs), np.hstack(ys)

train_datasets = [datasets[name] for name in trainset_names]
test_datasets = [datasets[name] for name in testset_names]
X_train, y_train = merge_datasets(train_datasets)
X_test, y_test = merge_datasets(test_datasets)

def fill_deep_steganalysis_data_dirs(*, X_train, y_train, X_test, y_test, root_dir='', X_val=None, y_val=None):
    train_path = f'{root_dir}/train_data_dir'
    test_path = f'{root_dir}/test_data_dir'
    val_path = f'{root_dir}/val_data_dir'

    output_dataset(X_train, y_train, f'{train_path}/cover', f'{train_path}/stego', reset_dirs=True)
    output_dataset(X_test, y_test, f'{test_path}/cover', f'{test_path}/stego', reset_dirs=True)
    if X_val is None or y_val is None:
        X_val, y_val = X_test, y_test
    
    output_dataset(X_val, y_val, f'{val_path}/cover', f'{val_path}/stego', reset_dirs=True)

# root_dir = '/mnt/exdisk2/image_steganalysis/deep_steganalysis'
fill_deep_steganalysis_data_dirs(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, root_dir='/mnt/exdisk2/image_steganalysis/deep_steganalysis')

added path: /home/danielg/danigil/ModelXRay


# Compile registry

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import compile_preprocessed_images_registry_pipeline
from model_xray.options import *
from model_xray.config_classes import *

model_names = model_collections['famous_le_100m'].union(model_collections['famous_le_10m'])
pretrained_model_configs = [PretrainedModelConfig(name=model_name, repo=ModelRepos.KERAS) for model_name in model_names]
compile_preprocessed_images_registry_pipeline(pretrained_model_configs=pretrained_model_configs)

added path: /home/danielg/danigil/ModelXRay


PyTorch version 2.4.0+cu118 available.
TensorFlow version 2.14.0 available.
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Initiating a new run for the pipeline: compile_preprocessed_images_registry_pipeline.
Uploading external artifact to 'external_artifacts/external_e85c68d0-9445-41cf-b287-3ff1f6b6998c'.
No materializer is registered for type <class 'model_xray.config_classes.PretrainedModelConfig'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'model_xray.config_classes.PretrainedModelConfig'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
No materializer is reg

PipelineRunResponse(body=PipelineRunResponseBody(created=datetime.datetime(2024, 9, 5, 13, 46, 44, 959530), updated=datetime.datetime(2024, 9, 5, 14, 32, 4, 982059), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 17, 23, 59, 37, 668468), updated=datetime.datetime(2024, 8, 18, 0, 2, 49, 626363), active=True, activation_token=None, full_name='', email_opted_in=False, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('8c66513b-5e16-4bbf-a271-7b389d6c1d2d'), permission_denied=False, name='default'), status=<ExecutionStatus.COMPLETED: 'completed'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2024, 8, 18, 0, 9, 44, 367644), updated=datetime.datetime(2024, 8, 20, 0, 9, 7, 782296), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 17, 23, 59, 37, 668468), updated=datetime.datetime(2024, 8, 18, 0, 2, 49, 626363), active=True, activation_token=None, full_name='', email_opted_in=False, is_

# Compile datasets

In [ ]:
from typing import Dict, List, Set
import os
import sys

import shutil

import numpy as np
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import compile_and_save_preprocessed_images_dataset_pipeline
from model_xray.options import *
from model_xray.config_classes import *
from zenml.client import Client

for run in compile_and_save_preprocessed_images_dataset_pipeline.model.get_runs():
    Client().delete_pipeline_run(run.id)

In [13]:
from zenml import Model


model = Model(
    name="preprocessed_images_classification_datasets",
    version="1.0",
)

model.delete_all_artifacts(only_link=False, delete_from_artifact_store=True, )

Deleted artifact '/mnt/exdisk2/zenml_artifact_store/custom_artifacts/mobilenets_train_xs023_x/d16d948d-ba20-4d18-8b62-340c8979afd0' from the artifact store.
Deleted artifact '/mnt/exdisk2/zenml_artifact_store/custom_artifacts/mobilenets_train_xs023_y/9de3cdca-9265-494b-b370-8fc346f5100c' from the artifact store.
Deleted artifact '/mnt/exdisk2/zenml_artifact_store/custom_artifacts/famous_le_100m_test_xs23_x/825aa51d-56f7-4041-aaff-2ea903b5a200' from the artifact store.
Deleted artifact '/mnt/exdisk2/zenml_artifact_store/custom_artifacts/famous_le_100m_test_xs23_y/1601b4ec-046e-46cf-b9ad-77c8b173b535' from the artifact store.
Deleted artifact '/mnt/exdisk2/zenml_artifact_store/custom_artifacts/famous_le_100m_test_benigns_x/a9a33b0b-8730-4a1c-b5e6-8fb77c6a27b8' from the artifact store.
Deleted artifact '/mnt/exdisk2/zenml_artifact_store/custom_artifacts/famous_le_100m_test_benigns_y/d09ee833-0051-41a9-8e9d-906b8caac9b0' from the artifact store.
Deleted artifact '/mnt/exdisk2/zenml_artifac

In [1]:
from typing import Dict, List, Set
import os
import sys

import shutil

import numpy as np
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import compile_and_save_preprocessed_images_dataset_pipeline
from model_xray.options import *
from model_xray.config_classes import *

curr_mc = 'famous_le_10m'

train_mzs, test_mzs = dataset_split[curr_mc]

payload_filepath: Optional[str] = None # '/mnt/exdisk2/model_xray/malware_payloads/m_77e05', None
embed_payload_type = PayloadType.BINARY_FILE if payload_filepath is not None else PayloadType.RANDOM


# imsize = 256
# imtype = ImageType.GRAYSCALE_THREEPART_WEIGHTED_AVG
imsize = 100
imtype = ImageType.GRAYSCALE_FOURPART

image_rep_config = ImageRepConfig.ret_image_rep_config(imtype)

image_preprocess_config=ImagePreprocessConfig(
    image_height=imsize,
    image_width=imsize,
)

from model_xray.utils.dataset_utils import *

added path: /home/danielg/danigil/ModelXRay


PyTorch version 2.4.0+cu118 available.
TensorFlow version 2.14.0 available.


In [2]:
get_dataset_name(mc="famous_le_10m_zoos", xs=[None,], imsize=256, imtype=ImageType.GRAYSCALE_FOURPART, ds_type='train', embed_payload_type=embed_payload_type, payload_filepath=payload_filepath)

'ds_type:train,imsize:256,imtype:grayscale_fourpart,mc:famous_le_10m_zoos,xs:0'

In [3]:
get_dataset_name(mc="famous_le_10m_zoos", xs=[None,15, 33, 3, 11, 30], imsize=256, imtype=ImageType.GRAYSCALE_FOURPART, ds_type='train', embed_payload_type=embed_payload_type, payload_filepath=payload_filepath)

'ds_type:train,embed_payload_type:binary_file,imsize:256,imtype:grayscale_fourpart,mc:famous_le_10m_zoos,payload_filepath:/mnt/exdisk2/model_xray/malware_payloads/m_77e05,xs:0|3|11|15|30|33'

In [2]:
pretrained_model_train_configs = get_pretrained_model_configs(train_mzs)
pretrained_model_test_configs = get_pretrained_model_configs(test_mzs)

test_benigns_dataset_name = get_dataset_name(
    mc=curr_mc,
    xs=[None,],
    imsize=imsize,
    imtype=imtype,
    ds_type='test',
)

compile_and_save_preprocessed_images_dataset_pipeline(
    dataset_name=test_benigns_dataset_name,
    pretrained_model_configs=pretrained_model_test_configs,
    x_values=[None,],
    image_rep_config=image_rep_config,
    image_preprocess_config=image_preprocess_config,
)

for x in range(1, 24):
    xs = [None, x]
    train_dataset_name = get_dataset_name(
        mc=curr_mc,
        xs=xs,
        imsize=imsize,
        imtype=imtype,
        ds_type='train',

        embed_payload_type=embed_payload_type,
        payload_filepath=payload_filepath,
    )

    compile_and_save_preprocessed_images_dataset_pipeline(
        dataset_name=train_dataset_name,
        pretrained_model_configs=pretrained_model_train_configs,
        # x_values=[None,] + [x for x in range(1, 24)],
        x_values=xs,
        image_rep_config=image_rep_config,
        image_preprocess_config=image_preprocess_config,

        payload_filepath=payload_filepath,
    )
    xs = [x,]

    test_dataset_name = get_dataset_name(
        mc=curr_mc,
        xs=xs,
        imsize=imsize,
        imtype=imtype,
        ds_type='test',

        embed_payload_type=embed_payload_type,
        payload_filepath=payload_filepath,
    )
    compile_and_save_preprocessed_images_dataset_pipeline(
        dataset_name=test_dataset_name,
        pretrained_model_configs=pretrained_model_test_configs,
        # x_values=[None,] + [x for x in range(1, 24)],
        x_values=xs,
        image_rep_config=image_rep_config,
        image_preprocess_config=image_preprocess_config,

        payload_filepath=payload_filepath,
    )


Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Initiating a new run for the pipeline: compile_and_save_preprocessed_images_dataset_pipeline.


Uploading external artifact to 'external_artifacts/external_499e920c-71be-49d0-bb0d-6896508b801f'.
No materializer is registered for type <class 'model_xray.config_classes.PretrainedModelConfig'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'model_xray.config_classes.PretrainedModelConfig'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
No materializer is registered for type <class 'model_xray.config_classes.PretrainedModelConfig'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'model_xray.config_

In [5]:
from typing import Dict, List, Set
import os
import sys

import shutil

import numpy as np
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import retreive_datasets
from model_xray.options import *
from model_xray.config_classes import *

mc = 'famous_le_10m'
train_x = 16
imsize = 100
imtype = ImageType.GRAYSCALE_FOURPART
payload_filepath = '/mnt/exdisk2/model_xray/malware_payloads/m_77e05'

trainset_name = get_dataset_name(
        mc=mc,
        ds_type='train',
        xs=[train_x, None],
        imsize=imsize,
        imtype=imtype,
        payload_filepath=payload_filepath,
    )

ret = retreive_datasets(
    [trainset_name,]
)

X_train, y_train = ret[trainset_name]

added path: /home/danielg/danigil/ModelXRay


In [6]:
X_train

array([[[121, 122, 123, ..., 118, 120, 121],
        [123, 124, 124, ..., 122, 121, 119],
        [125, 128, 126, ..., 124, 119, 118],
        ...,
        [128, 127, 128, ..., 127, 129, 127],
        [125, 128, 128, ..., 131, 129, 129],
        [129, 126, 124, ..., 124, 124, 124]],

       [[121, 122, 123, ..., 118, 120, 121],
        [123, 124, 124, ..., 122, 121, 119],
        [125, 128, 126, ..., 124, 119, 118],
        ...,
        [100, 100, 101, ...,  96,  98,  98],
        [100,  99, 100, ...,  97,  99,  97],
        [100,  96,  94, ...,  95,  95,  96]],

       [[118, 119, 120, ..., 119, 117, 122],
        [122, 124, 124, ..., 120, 120, 121],
        [122, 120, 121, ..., 119, 118, 121],
        ...,
        [128, 127, 128, ..., 129, 129, 127],
        [127, 127, 128, ..., 128, 129, 125],
        [126, 127, 126, ..., 126, 126, 127]],

       [[118, 119, 120, ..., 119, 117, 122],
        [122, 124, 124, ..., 120, 120, 121],
        [122, 120, 121, ..., 119, 118, 121],
        ..

In [7]:
y_train

array([0, 1, 0, 1, 0, 1])